# **EDA - Exploratory Data Analysis**

In [ ]:
import pandas as pd 
import pycountry
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path # manage paths for the project

In [ ]:
# define the path of the data
data_path = Path('..') / 'data' / 'raw' / 'candidates.csv'

In [ ]:
# first trouble, the data is not separated by comma, instead, it is separated 
# by semicolon so, the argument (sep='<separator>') allows indicate to pandas 
# the separation method

df_candidates_org = pd.read_csv(data_path, sep=';')
df_candidates_org

In [ ]:
df_candidates_org.info()

# No null values

In [ ]:
# making a copy for temporal transformations

df_candidates = df_candidates_org.copy()
df_candidates

In [ ]:
# to make easier the EDA process, all str values in df (including columns 
# names) were changed for their lowercase version 

df_candidates.columns = df_candidates.columns.str.lower()
df_candidates

In [ ]:
text_colummns = df_candidates.select_dtypes(include=['object', 
                                                    'string']).columns

for column in text_colummns:
    if column != "email":
        df_candidates[column] = df_candidates[column].str.lower()
    
df_candidates.head()

In [ ]:
df_candidates.describe()

# Numerical values dont show out of range

In [ ]:
# To confirm that the numerical values are within expected ranges and 
# understand their distribution, we plot histograms. The almost uniform 
# distribution of scores suggests the data might be synthetically generated.

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.histplot(df_candidates["yoe"], 
            bins=15, ax=axes[0], 
            color="skyblue").set_title("Distribution of YOE")
sns.histplot(df_candidates["code challenge score"], 
            bins=10, ax=axes[1], 
            color="lightgreen").set_title("Code Challenge Score")
sns.histplot(df_candidates["technical interview score"], bins=10, ax=axes[2], 
            color="salmon").set_title("Technical Interview Score")

plt.tight_layout()
plt.show()

In [ ]:
# now, in necessary verify the unique values for de categorical columns 

df_candidates["email"].unique()

# The “length” section indicates that only 49,833 records are unique.

In [ ]:
# To decide what to do, it will be investigated whether these are data errors 
# or whether the context of the problem allows it 

# filter which columns that had emails duplicated
df_duplicated = df_candidates[df_candidates.duplicated(subset= ["email"])] 
df_duplicated

In [ ]:
duplicated_emails = df_duplicated["email"].values
print(duplicated_emails)

In [ ]:
# run this celd if you wanna see all records
pd.set_option('display.max_rows', None)

In [ ]:
# run this celd if you dont wanna see all records
pd.reset_option('display.max_rows')

In [ ]:
df_candidates[df_candidates["email"].isin(duplicated_emails)].sort_values(
                                                                by="email")

In [ ]:
# While it is true that other contact details (such as a phone number) could 
# be used to uniquely identify a candidate, the fact that the same email 
# address is associated with more than one person, from a data quality 
# perspective, would pose a risk to the reliability of that data. This could 
# be due to fraud or identity theft, an error in the data collection system 
# (such as a faulty web form), or corrupted data at source. Doubtful data s
# hould not be entered into the data warehouse. Therefore, all records with the
# same email address will be deleted.

df_candidates.drop_duplicates(subset=['email'], keep=False)


In [ ]:
# Verifying the date range

print(f"fecha inicial: {df_candidates['application date'].min()}")
print(f"fecha final: {df_candidates['application date'].max()}")

# Theres no dates out of range

In [ ]:
df_candidates["country"].unique()

# 244 unique countries were identified, which exceeds the 195 sovereign states.
# This is due to the inclusion of dependent territories (e.g., Cocos Islands) 
# according to the ISO standard. For this exercise, they will be assumed as 
# valid geographic locations for candidates.

In [ ]:

# * However, a small verification will be implemented with the help of the 
# * pycountry library.

for country in df_candidates["country"]:
    try:
        country = pycountry.countries.get(name=country)
    except LookupError:
        print(country)
        
# ! All countries are valid.

In [ ]:

# ? Consideration is being given to reducing the number of categories to 
# ? facilitate analysis (e.g., “inter” and “trainee”). 

df_candidates["seniority"].unique()

In [ ]:
# One finding was that there were records with a YOE of 27 but whose seniority
# was trainee. It is understood that the data was generated synthetically, but 
# in a real context, years of experience are correlated with seniority. During 
# the transformation stage, logic will be implemented to recalculate seniority 
# based on years of experience. 

# TODO: in the transformation phase, change values of seniority by the yoe values

df_candidates[(df_candidates["yoe"]>25) & (df_candidates["seniority"] == "intern")]

# there is 1209 candidates with an "intern" seniority and more than 25 yoe, 
# don't make sense in this context


In [ ]:
# To understand the internal consistency of the profiles, we cross-reference 
# Years of Experience (YOE) with Seniority Level. The plot below reveals a 
# critical anomaly: the distribution boxes are virtually identical for all 
# roles (from Intern to Architect). This proves a lack of correlation and 
# justifies the need for a transformation rule in the ETL process to 
# recalculate Seniority based on YOE.

plt.figure(figsize=(10, 6))
# Define a logical order for the x-axis based on the unique values
order = ["trainee", "intern", "junior", "mid-level", "senior", "lead", 
        "architect"]

sns.boxplot(data=df_candidates, x="seniority", y="yoe", order=order, 
        hue="seniority", palette="Set2", legend=False)

plt.title("Anomaly Detection: Years of Experience vs Seniority Level")
plt.xlabel("Seniority Level")
plt.ylabel("Years of Experience (YOE)")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Nothing to do here

df_candidates["technology"].unique()

In [ ]:
# Given the high cardinality of the technology column, we identify the top 10 
# most frequent profiles to understand the volume and focus of the candidate 
# pool.

plt.figure(figsize=(10, 6))

top_tech = df_candidates["technology"].value_counts().head(10)
sns.barplot(x=top_tech.values, y=top_tech.index, hue=top_tech.index, 
            palette="viridis", legend=False)

plt.title("Top 10 Technologies Applied For")
plt.xlabel("Number of Candidates")
plt.ylabel("Technology")
plt.show()

In [ ]:
# In a real context, usually a candidate with a high score 
# in code tends to have a lower score in the interview. Since the data is 
# synthetic, there should be no correlation, so with the code below we should 
# see a number close to 0.

df_candidates[['code challenge score', 'technical interview score']].corr()

In [ ]:
# To understand if there is any underlying relationship between our 
# numeric variables (Years of Experience, Code Challenge Score, and Technical 
# Interview Score), a correlation matrix was generated. In a synthetic dataset, 
# its expected that these values to be close to 0, indicating a lack of 
# correlation.

plt.figure(figsize=(8, 6))

# Select only numeric columns for correlation
numeric_cols = ["yoe", "code challenge score", "technical interview score"]
correlation_matrix = df_candidates[numeric_cols].corr()

# Create a heatmap
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1, fmt=".2f", linewidths=.5)

plt.title("Correlation Matrix of Numeric Variables")
plt.show()